<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [11]:
import pandas as pd
import psycopg2
import warnings

warnings.filterwarnings('ignore')

In [12]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [13]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса

query = f'''select * 
                from public.vacancies
             '''
vacancies = pd.read_sql_query(query, connection)

In [5]:
vacancies.shape[0]

49197

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
query = f'''select * 
                from public.employers
             '''
employers = pd.read_sql_query(query, connection)

In [7]:
employers.shape[0]

23501

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
query = f'''select * 
                from public.areas
             '''
areas = pd.read_sql_query(query, connection)

In [9]:
areas.shape[0]

1362

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
query = f'''select * 
                from public.industries
             '''
industries = pd.read_sql_query(query, connection)

In [11]:
industries.shape[0]

294

В базе данных присутствуют 49197 от 23501 работодателей по 1362 регионам и 294 специальностям.

В среднем, каждый работодатель выкладывает чуть больше чем две вакансии.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
query = f'''select a.name areas, count(area_id) cnt
            from public.vacancies v
            join public.areas a on a.id = area_id
            group by areas
            order by cnt desc
        '''
cnt_vacancies = pd.read_sql_query(query, connection)

In [13]:
cnt_vacancies.head(15)

,areas,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
query = f'''select count(id) cnt
            from public.vacancies
            where salary_from is not null or salary_to is not null

        '''
cnt_salary_not_null = pd.read_sql_query(query, connection)

In [15]:
int(cnt_salary_not_null.loc[0])

24073

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
query = f'''select avg(salary_from) salary_from, avg(salary_to) salary_to
            from public.vacancies


        '''
salary_difference = pd.read_sql_query(query, connection)

In [17]:
salary_from_arg = round(float(salary_difference.iloc[:,0]))
salary_to_arg = round(float(salary_difference.iloc[:,1]))
display(salary_from_arg)
salary_to_arg

71065

110537

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
query = f'''select schedule, employment, count(id)
            from public.vacancies
            group by employment, schedule
            order by count(id) desc

        '''
schedule_employment = pd.read_sql_query(query, connection)

In [19]:
schedule_employment

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
query = f'''select experience, count(id)
            from public.vacancies
            group by experience
            order by count(id)

        '''
experience_cnt = pd.read_sql_query(query, connection)

In [21]:
experience_cnt

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Самое большое количество вакансий располагается в столицах и городах. Москве 5333, Санкт-Петербург 2851, Минск 2112, Новосибирск 2006, Алматы 1892

Из 49197 вакансий только у 24073 заполнено хотя бы одно из двух полей с зарплатой. Меньше половины работодателей предпочитают указывать ЗП. Нижняя граница средней зарплаты 71065. Верхняя граница средней зарплаты 110537.

Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях.

Абсолютное большинство из 49197 вакансий это полная занятость и полный рабочий день (35367 вакансий) либо полная занятость удаленная работа (7802 вакансий). Все остальные варианты сочетаний типов трудоустройства и типов рабочего графика встречаются во много раз реже.

Требуемый опыт работы (experience) в порядке возрастания количества вакансий
Чаще всего работодатели ищут специалистов с небольшим опытом работы, от 1 до 3 лет, 26152 вакансий.  На втором месте вакансии с требованием опыта работы от 3 до 6 лет 14511 вакансий. Следующими по количеству стоят вакансии без требований к опыту работы 7197. Их 3,63 раза меньше чем специалистов с опытом работы от 1 года. На последнем месте в плане распространенности специалисты с опытом работы от 6 лет.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
query = f'''(select e.name , count(v.id) count_
            from public.vacancies v
            join public.employers e on v.employer_id = e.id
            group by e.name
            order by count(v.id) desc
            limit 1)

            union

            (select e.name, count(v.id)
            from public.vacancies v
                join public.employers e on v.employer_id = e.id
            group by e.name
            order by count(v.id) desc
            offset 4 limit 1)
            order by count_ desc
        '''
employers_count = pd.read_sql_query(query, connection)

In [23]:
employers_count

,name,count_
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [15]:
query = f'''select distinct a.name, count(distinct e.id) count_employers, count(v.id) count_vacancies
            from public.areas a
                left join public.employers e on e.area = a.id
                left join public.vacancies v on v.area_id = a.id
            group by a.name
            order by count_vacancies, count_employers desc
        '''
zero_employers_count = pd.read_sql_query(query, connection)

In [25]:
zero_employers_count.iloc[0,0]

'Россия'

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [26]:
query = f'''select e.name, count(distinct v.area_id) 
            from public.employers e
                left join public.vacancies v on v.employer_id = e.id
            group by e.name 
            order by count(v.area_id) desc

        '''
employers_vac_cnt = pd.read_sql_query(query, connection)

In [27]:
employers_vac_cnt

,name,count
0,Яндекс,181
1,Ростелеком,152
2,Тинькофф,43
3,СБЕР,24
4,Газпром нефть,24
...,...,...
23170,Независимое агентство Аналитика товарных рынков,0
23171,Независимое Информационное Агентство,0
23172,Независимое телевидение Севастополя,0
23173,Независимость,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [28]:
query = f'''select count(distinct e.id)
            from public.employers e
            left join public.EMPLOYERS_INDUSTRIES ei on e.id = ei.employer_id
            where ei.employer_id is null
        '''
empl_ind_null_cnt = pd.read_sql_query(query, connection)

In [29]:
int(empl_ind_null_cnt.iloc[0,0])

8419

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [30]:
query = f'''select e.name, count(distinct ei.industry_id)
            from public.employers e
                left join public.EMPLOYERS_INDUSTRIES ei on e.id = ei.employer_id
            group by e.name
            having count(distinct ei.industry_id) =4
            order by e.name
            offset 2 limit 1
        '''
empl_ind_cnt_4 = pd.read_sql_query(query, connection)

In [31]:
empl_ind_cnt_4.iloc[0,0]

'2ГИС'

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [32]:
query = f'''select count(distinct e.id)
            from public.employers e
                left join public.EMPLOYERS_INDUSTRIES ei on e.id = ei.employer_id
                left join public.INDUSTRIES i on ei.industry_id = i.id 
            where i.name = 'Разработка программного обеспечения'
        '''
empl_cnt_7540 = pd.read_sql_query(query, connection)

In [33]:
int(empl_cnt_7540.iloc[0,0])

3553

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [34]:
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' # Определяем адрес страницы
response = requests.get(url)  # Выполняем GET-запрос
page = BeautifulSoup(response.text, 'html.parser')
list_city_million = pd.read_html(url)[1]
list_city_million['Город'] = list_city_million['Город'].apply(lambda x:x.replace('[a]',''))

In [35]:
query = f'''select 
            a.name, count(v.id)
            from public.areas a
            left join public.vacancies v on v.area_id = a.id
            left join public.employers e on v.employer_id = e.id
            where e.name = 'Яндекс'
            group by a.name
        '''
pd_city = pd.read_sql_query(query, connection)

dt_city_merge = list_city_million.merge(
    pd_city,
    left_on='Город',
    right_on='name',
    how='left'
)


In [36]:
dt_city_count = dt_city_merge[['Город','count']]

dt_city_count.loc[16] = ['Total:', sum(dt_city_count['count'])]
dt_city_count.sort_values(by='count')

,Город,count
11,Омск,21
6,Челябинск,22
7,Красноярск,23
15,Волгоград,24
4,Казань,25
10,Ростов-на-Дону,25
14,Пермь,25
8,Самара,26
9,Уфа,26
12,Краснодар,30


***

3553 из 23501 указывают сферу своей деятельности разработку программного обеспечения
8419 работодателей не указывает сферу свой деятельности.

Яндекс размещает самое большое количество вакансий среди всех работодателей. 1933 из 49197. Так же, Яндекс размещает вакансии в самом большом количестве регионов (181 регион). Преимущественно Яндекс размещает вакансии в крупных городах, таких как Москва (54), Санкт-Петербург (42), Екатеринбург (39), Нижний Новгород (36) Новосибирск (35)

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [37]:
query = f'''select count(id)
            from public.vacancies 
            where lower(name) like '%data%' or lower(name) like '%данн%'
        '''
data_cnt_vacancies = pd.read_sql_query(query, connection)

In [38]:
int(data_cnt_vacancies.loc[0])

1771

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [39]:
query = f'''select count(id)
            from public.vacancies 
            where
            (lower(name) like '%data science%'    
            or lower(name) like '%data scientist%' 
            or lower(name) like '%исследователь данных%'
            or (lower(name) like '%ml%' and lower(name) not like '%html%')
            or lower(name) like '%machine learning%'
            or lower(name) like '%машинн%обучен%'
            )
            and
            (
            lower(experience) like '%нет опыта%'
            or lower(name) like '%junior%'
            or lower(employment) like '%стажировка%'
            )
        '''
data_scince_cnt_vac = pd.read_sql_query(query, connection)

In [40]:
int(data_scince_cnt_vac.loc[0])

51

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [41]:
query = f'''select count(id)
            from public.vacancies 
            where
            (lower(name) like '%data science%'
            or lower(name) like '%data scientist%' 
            or lower(name) like '%исследователь данных%'
            or (name like '%ML%' and name not like '%HTML%')
            or lower(name) like '%machine learning%'
            or lower(name) like '%машинн%обучен%')
    
            and
    
            (key_skills like '%SQL%' 
            or lower(key_skills) like '%postgres%')
        '''
        
ds_sql_cnt_vac = pd.read_sql_query(query, connection)

In [42]:
int(ds_sql_cnt_vac.loc[0])

201

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [43]:
query = f'''select count(id)
            from public.vacancies 
            where
            (lower(name) like '%data science%'
            or lower(name) like '%data scientist%' 
            or lower(name) like '%исследователь данных%'
            or (name like '%ML%' and name not like '%HTML%')
            or lower(name) like '%machine learning%'
            or lower(name) like '%машинн%обучен%'
            )
            and 
            lower(key_skills) like '%python%'
        '''
        
ds_python_cnt_vac = pd.read_sql_query(query, connection)

In [44]:
int(ds_python_cnt_vac.loc[0])

351

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [45]:
query = f'''select
            avg(length(key_skills) - length(replace(key_skills, CHR(9), '')) +1)
            from
            public.vacancies /*из таблицы вакансий*/
            where (lower(name) like '%data scientist%'
            or lower (name) like '%data science%'
            or lower (name) like '%исследователь данных%'
            or (name like '%ML%' and name not like '%HTML%')
            or lower (name) like '%machine learning%'
            or lower (name) like '%машинн%обучен%%')
        
            and 
            
             key_skills is not null
        '''
               
avg_key_cnt = pd.read_sql_query(query, connection)        


In [46]:
round(float(avg_key_cnt.loc[0]),2)

6.41

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [47]:
query = f'''select
            experience, avg((coalesce(salary_from, salary_to) + coalesce(salary_to,salary_from))/2)
            from
            public.VACANCIES v /*из таблицы вакансий*/
            where (lower(v.name) LIKE '%data scientist%'
            or lower (v.name) LIKE '%data science%'
            or lower (v.name) LIKE '%исследователь данных%'
            or (v.name LIKE '%ML%' and v.name not LIKE '%HTML%')
            or lower (v.name) LIKE '%machine learning%'
            or lower (v.name) LIKE '%машинн%обучен%%')
    
            and
            (salary_from is not null or salary_to is not null)
            group by experience
        '''
               
avg_salary_exp = pd.read_sql_query(query, connection)   

In [48]:
avg_salary_exp

,experience,avg
0,Нет опыта,74642.857143
1,От 1 года до 3 лет,139674.750000
2,От 3 до 6 лет,243114.666667


In [49]:
round(float(avg_salary_exp.iloc[2,1]))

243115

***

Из 49197 вакансий только 1771 (2,93%) относится к данным. И только 51 - вакансии data аналитиков уровня junior (0,10%).
201 вакансий требуют ключевого навыка SQL или postgres (0,40%).
351 вакансий требуют ключевого навыка python (0.71 %).

В анкетах в среднем 6.41 ключевых навыков.

Средняя заработная плата для вакансий без опыта 74643 руб., с опытом от 1 до 3 лет 139675 руб. С опытом от 3 до 6 лет 243115 руб.

# Общий вывод по проекту

## Дополнительные запросы
___

Количество вакансий Data специалистов по городам

In [50]:
query = f'''select
                a.name "Расположение", count(v.id) "Количество вакансий"
            from
                public.VACANCIES v
                join public.areas a on v.area_id = a.id
            where 
                (lower(v.name) LIKE '%data scientist%'
                or lower (v.name) LIKE '%data science%'
                or lower (v.name) LIKE '%исследователь данных%'
                or (v.name LIKE '%ML%' and v.name not LIKE '%HTML%')
                or lower (v.name) LIKE '%machine learning%'
                or lower (v.name) LIKE '%машинн%обучен%%')
            group by a.name
            order by "Количество вакансий" desc
        '''
               
ds_area_count = pd.read_sql_query(query, connection)   
ds_area_count.head(15)

,Расположение,Количество вакансий
0,Москва,217
1,Санкт-Петербург,64
2,Новосибирск,23
3,Нижний Новгород,20
4,Казань,16
5,Алматы,16
6,Минск,9
7,Томск,8
8,Екатеринбург,7
9,Краснодар,6


Распределение вакансий, связанных с data scince, отличается от распределения общего количества вакансий. Больше вакансий сосредоточено в Москве и крупных городах России. 

___

Количество вакансий data специалистов по компаниям

In [51]:
query = f'''select
                e.name "Компании", count(v.id) "Количество вакансий"
            from
                public.VACANCIES v
                join public.employers e on v.employer_id = e.id
            where 
                (lower(v.name) LIKE '%data scientist%'
                or lower (v.name) LIKE '%data science%'
                or lower (v.name) LIKE '%исследователь данных%'
                or (v.name LIKE '%ML%' and v.name not LIKE '%HTML%')
                or lower (v.name) LIKE '%machine learning%'
                or lower (v.name) LIKE '%машинн%обучен%%')
            group by e.name
            order by "Количество вакансий" desc
        '''
               
cnt_vacancies_ds_empl = pd.read_sql_query(query, connection)   
cnt_vacancies_ds_empl.head(15)

,Компании,Количество вакансий
0,СБЕР,37
1,Bell Integrator,25
2,Банк ВТБ (ПАО),18
3,VK,15
4,Positive Technologies,11
5,EvenBet Gaming,9
6,Яндекс,9
7,МегаФон,8
8,Andersen,7
9,Ozon,6


Больше всего Data специалистов ищут СБЕР, Bell Integrator, Банк ВТБ (ПАО), VK, Positive Technologies. В целом какая-то четкая закономерность не усматривается.

___
Количество вакансий junior data специалистов по компаниям

In [52]:
query = f'''select
                e.name "Компании", count(v.id) "Количество вакансий junior"
            from
                public.VACANCIES v
                join public.employers e on v.employer_id = e.id
            where 
                (lower(v.name) LIKE '%data scientist%'
                or lower (v.name) LIKE '%data science%'
                or lower (v.name) LIKE '%исследователь данных%'
                or (v.name LIKE '%ML%' and v.name not LIKE '%HTML%')
                or lower (v.name) LIKE '%machine learning%'
                or lower (v.name) LIKE '%машинн%обучен%%')

                and
                (lower(experience) like '%нет опыта%'
                or lower(v.name) like '%junior%'
                or lower(employment) like '%стажировка%')
            
            group by e.name, v.employment
            order by "Количество вакансий junior" desc
        '''
               
cnt_vacancies_ds_junior = pd.read_sql_query(query, connection)   
cnt_vacancies_ds_junior.head(10)

'стажировка'

,Компании,Количество вакансий junior
0,Банк ВТБ (ПАО),10
1,The Skolkovo Institute of Science and Technology,3
2,Comexp,3
3,SteadyControl,2
4,Диплей,2
5,Seldon,1
6,Theyr Ltd,1
7,VK,1
8,Алгоритмические Системы,1
9,АЛЬФА-БАНК,1


Вакансий Junior data специалистов в несколько раз меньше чем вакансий data специалистов с большим опытом.

Преимущественно их ищет Банк ВТБ (ПАО), The Skolkovo Institute of Science and Technology, Comexp, Диплей    , СтарЛайн, Научно-производственное объединение, SteadyControl.

___

Верхняя и нижняя граница средней заработной платы у Data специалистов

In [5]:
query = f'''select
                avg(salary_from) salary_from, 
                avg(salary_to) salary_to
            from
                public.VACANCIES v
                join public.employers e on v.employer_id = e.id
            where 
                (lower(v.name) LIKE '%data scientist%'
                or lower (v.name) LIKE '%data science%'
                or lower (v.name) LIKE '%исследователь данных%'
                or (v.name LIKE '%ML%' and v.name not LIKE '%HTML%')
                or lower (v.name) LIKE '%machine learning%'
                or lower (v.name) LIKE '%машинн%обучен%%')
        '''
               
avg_salary_ds = pd.read_sql_query(query, connection)   
avg_salary_ds

,salary_from,salary_to
0,146804.521739,216355.772727


___

Средняя заработная плата junior по работодателям

In [53]:
query = f'''select 
                e.name "Название компании", avg((coalesce(salary_from, salary_to) + coalesce(salary_to,salary_from))/2) "Средняя ЗП"
            from public.vacancies v
                join public.employers e on v.employer_id = e.id
            where
                (lower(v.name) like '%data science%'    
                or lower(v.name) like '%data scientist%' 
                or lower(v.name) like '%исследователь данных%'
                or (lower(v.name) like '%ml%' and lower(v.name) not like '%html%')
                or lower(v.name) like '%machine learning%'
                or lower(v.name) like '%машинн%обучен%')
    
            and
                (lower(experience) like '%нет опыта%'
                or lower(v.name) like '%junior%'
                or lower(employment) like '%стажировка%')
    
            and
                (v.salary_from is not null or v.salary_to is not null)
            group by e.name
            order by "Средняя ЗП" desc
        '''
               
avg_salary_ds_junior = pd.read_sql_query(query, connection)   
avg_salary_ds_junior.head(10)

,Название компании,Средняя ЗП
0,Университет ИТМО,115000.0
1,ННФормат,98000.0
2,Апэрбот,80000.0
3,Строительный Двор,72500.0
4,DNS Головной офис,70000.0
5,Випакс+,60000.0
6,Улыбка радуги,50000.0
7,Theyr Ltd,50000.0
8,КБ Уральский банк реконструкции и развития (УБ...,37000.0


Из 34 организаций, которые ищут junior data специалистов, зарплату указывают только 8.  Но заработная плата у junior существенно выше чем средняя или медианная заработная плата в России.

___

Средняя заработная плата Data специалистов в зависимости от расположения.

In [7]:
query = f'''select
                a.name "Расположение", 
                avg((coalesce(salary_from, salary_to) + coalesce(salary_to,salary_from))/2) "Средняя ЗП"
            from
                public.VACANCIES v
                join public.areas a on v.area_id = a.id
            where (lower(v.name) LIKE '%data scientist%'
                or lower (v.name) LIKE '%data science%'
                or lower (v.name) LIKE '%исследователь данных%'
                or (v.name LIKE '%ML%' and v.name not LIKE '%HTML%')
                or lower (v.name) LIKE '%machine learning%'
                or lower (v.name) LIKE '%машинн%обучен%%')
                and
                (salary_from is not null or salary_to is not null)
            group by a.name
            order by "Средняя ЗП" desc
        '''
               
ds_area_salary = pd.read_sql_query(query, connection)
ds_area_salary.head(15)

,Расположение,Средняя ЗП
0,Кипр,300000.000000
1,Армения,268862.666667
2,Черногория,233794.000000
3,Турция,233794.000000
4,Сербия,233794.000000
5,Москва,207629.629630
6,Белгород,200000.000000
7,Санкт-Петербург,173335.142857
8,Новосибирск,166333.333333
9,Рязань,136666.666667


In [9]:
sum(ds_area_salary.iloc[0:4,1])/len(ds_area_salary.iloc[0:4,1])

259112.6666666667

Средняя заработная плата Data специалистов в Москве - самая большая в России, но в компаниях, располагающихся за рубежом встречаются зарплаты сильно выше.

С высокой долей вероятности, учитывая географию зарубежных вакансий, данные вакансии размещены компаниями, которые когда-то релоцировались из России.

___
На основе имеющихся данных можно сделать следующие выводы:

<br>

Представленные вакансии имеют существенно более высокую заработную плату по сравнению с
медианной ЗП в России, которая в 2021 году составляла 35000 рублей. Для сравнения нижняя граница средней ЗП в вакансиях равна 71065. Верхняя граница 110537. При этом нижняя граница средней зарплаты Data специалистов равна 146805 руб. Верхняя граница 216356 руб.

Работа в сфере Data scince оплачивается очень высоко даже в сравнении с другими IT специальностями. Начинающий дата аналитик может рассчитывать на ЗП в районе 74643 руб. С опытом от 1 до 3 лет средняя ЗП вырастает на 87% до 139675.  С опытом от 3 до 5 лет средняя ЗП вырастает еще на 74 % до 243115 руб.

Средняя заработная плата Data специалистов в Москве - самая большая в России, но в компаниях, располагающихся за рубежом встречаются зарплаты сильно выше. Для сравнения средняя ЗП data специалиста в Москве 207629 руб. Но средняя ЗП в зарубежных компаниях, стоящих выше в сортировке по ЗП, составила 259113 руб.

Учитывая географию этих вакансий, и тот факт, что они ищут русских специалистов на российской площадке, данные вакансии размещены компаниями, которые когда-то релоцировались из России. Что указывает на дополнительные перспективы в данной работе.

<br>

Абсолютное большинство из 49197 вакансий это полная занятость и полный рабочий день (35367 вакансий) либо полная занятость удаленная работа (7802 вакансий). Все остальные варианты сочетаний типов трудоустройства и типов рабочего графика встречаются во много раз реже.

Самое большое количество вакансий относятся к столицам и большим городам. Как пример, топ 5 городов по количеству вакансий: Москве 5333, Санкт-Петербург 2851, Минск 2112, Новосибирск 2006, Алматы 1892.

Распределение по городам вакансий Data специалистов отличается от общего числа вакансий. Так основное количество вакансий DS располагается в Москве и крупных городах. Как пример, топ 5 городов по количеству вакансий DS: Москва 217, Санкт-Петербург 64, Новосибирск 23, Нижний Новгород 20, Казань 16.

Запрос количества вакансий DS по компаниям не показал четкой взаимосвязи между компаниями, что может говорить о том, что data специалисты востребованы в большинстве IT компаний.

<br>

Вакансий Junior data специалистов в несколько раз меньше чем вакансий data специалистов с опытом от 1 года. И ,несмотря на то, что основная масса вакансий, это специалисты с небольшим опытом работы, главной проблемой может стать нахождение перовой работы, которая позволит получить опыт от 1 года.

Из 34 организаций, которые ищут junior data специалистов, зарплату указывают только 8. Что может свидетельствовать о том, что условия по зарплате у Junior DS могут сильно варьироваться.

<br>

Отдельно очень любопытные данные по требованиям к ключевым навыкам. Несмотря на то что в среднем в вакансии присутствует 6-7 ключевых навыков, из 1771 DS вакансий, только 351 (20%) вакансия имела требования к знанию python, и только 201 (11%) вакансия имела требования к знанию SQL или postgress. При чем часть этих вакансий не имела отношения к DS специальностям. Что внушает сомнения в релевантности данных из столбца key_skills.